# **Imports**

In [1]:
%%capture
'''!pip install numpy==1.26.4
!pip install pandas
!pip install matplotlib
!pip install nltk
!pip install git+https://github.com/antonio258/npp.git
!python -m spacy download pt_core_news_sm'''


In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gabrielprenassi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
import re
from unicodedata import normalize
from pre_processing import PreProcessing
import numpy as np
import spacy as sp
import matplotlib.pyplot as plt

# **Data**

In [4]:
df = pd.read_csv('../data/SUS_Simulador_Aposentadoria.csv')
df = df.dropna(subset=['Você conhece e já utilizou o Simulador de Aposentadoria no SOUGOV.BR?']).reset_index(drop=True)
df

,ID,Você conhece e já utilizou o Simulador de Aposentadoria no SOUGOV.BR?,Acho que gostaria de utilizar o Simulador de Aposentadoria com frequência,1\nAcho que gostaria de utilizar o Simulador de Aposentadoria com frequência,Eu achei o Simulador de Aposentadoria muito complexo,2\nEu achei o Simulador de Aposentadoria muito complexo,Eu achei o Simulador de Aposentadoria fácil de usar,3\nEu achei o Simulador de Aposentadoria fácil de usar,Eu acho que precisaria de ajuda de uma pessoa com conhecimentos específicos para usar o Simulador de Aposentadoria,4\nEu acho que precisaria de ajuda de uma pessoa com conhecimentos específicos para usar o Simulador de Aposentadoria,...,Eu achei o Simulador de Aposentadoria difícil de usar,8\nEu achei o Simulador de Aposentadoria difícil de usar,Eu me senti confiante usando o Simulador de Aposentadoria,9\nEu me senti confiante usando o Simulador de Aposentadoria,"Eu precisei aprender várias coisas novas antes de conseguir usar o Simulador de Aposentadoria. ,opcao:","10\nEu precisei aprender várias coisas novas antes de conseguir usar o Simulador de Aposentadoria. ,opcao:2","Agradeço a sua participação e abro o espaço para que você possa contribuir com alguma crítica, sugestão ou elogio sobre o Simulador de Aposentadoria.","Some a pontuação total de todas as questões ímpares (questões 1, 3, 5, 7 e 9) e, em seguida, subtraia 5 do total para obter (X).","Some a pontuação total de todas as questões de número par (questões 2, 4, 6, 8 e 10), então subtraia esse total de 25 para obter (Y).","Some a pontuação total dos novos valores (X+Y) e multiplique por 2,5."
0,3,Sim,Concordo totalmente,5,Discordo totalmente,1,Concordo totalmente,5,Discordo totalmente,1,...,Discordo totalmente,1,Discordo totalmente,1,Discordo totalmente,1,NaN,16,20,90
1,9,Sim,Discordo parcialmente,2,Concordo totalmente,5,Discordo totalmente,1,Concordo totalmente,5,...,Concordo totalmente,5,Discordo totalmente,1,"Não concordo, nem discordo",3,NaN,1,4,"12,5"
2,13,Sim,Concordo parcialmente,4,"Não concordo, nem discordo",3,"Não concordo, nem discordo",3,"Não concordo, nem discordo",3,...,Concordo parcialmente,4,Discordo parcialmente,2,Concordo totalmente,5,NaN,8,7,"37,5"
3,15,Sim,Concordo parcialmente,4,Concordo parcialmente,4,Concordo parcialmente,4,Discordo parcialmente,2,...,Concordo totalmente,5,Concordo totalmente,5,"Não concordo, nem discordo",3,NaN,17,10,"67,5"
4,16,Sim,Concordo totalmente,5,Concordo parcialmente,4,Discordo parcialmente,2,Discordo totalmente,1,...,Concordo totalmente,5,Concordo parcialmente,4,Discordo totalmente,1,"Ótima iniciativa, mas não funcionou comigo",12,9,"52,5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2250,4858,Sim,Concordo parcialmente,4,Concordo parcialmente,4,Discordo parcialmente,2,Concordo parcialmente,4,...,Concordo parcialmente,4,Discordo parcialmente,2,"Não concordo, nem discordo",3,Incluir um módulo relativo ao servidor com def...,8,7,"37,5"
2251,4859,Sim,"Não concordo, nem discordo",3,Concordo parcialmente,4,Discordo parcialmente,2,Concordo parcialmente,4,...,Concordo parcialmente,4,Discordo parcialmente,2,"Não concordo, nem discordo",3,NaN,7,7,35
2252,4860,Sim,Concordo totalmente,5,Discordo totalmente,1,Concordo totalmente,5,Discordo totalmente,1,...,Discordo totalmente,1,Concordo totalmente,5,Discordo totalmente,1,NaN,20,20,100
2253,4861,Sim,Concordo parcialmente,4,Discordo parcialmente,2,"Não concordo, nem discordo",3,"Não concordo, nem discordo",3,...,Discordo parcialmente,2,"Não concordo, nem discordo",3,"Não concordo, nem discordo",3,NaN,11,14,"62,5"


* **Preprocessing the text column**

In [5]:
stem = sp.load("pt_core_news_sm")
pp = PreProcessing(language="pt")

In [6]:
stopwords_dataset = [line.strip() for line in open('stopwords.txt', 'r').readlines()]
pp.append_stopwords_list(list(set(stopwords.words('portuguese')) - set(pp.stopwords))+stopwords_dataset)

In [7]:
def remove_repetion_caracteres(string, max_repetition=2):
    if not string:
        return string
    
    result = string[0]
    count = 1
    
    for i in range(1, len(string)):
        if string[i] == string[i-1]:
            count += 1
            if count <= max_repetition:
                result += string[i]
        else:
            count = 1
            result += string[i]
    
    return result

def preprocessing(text):

   if pd.isna(text):
        return np.nan

   tokens = stem(text.lower())
   text = ' '.join([text for token in tokens for text in token.lemma_.strip().split()]) # lemmatizing the text
   text = pp.remove_stopwords(text) # removes stopwords from the text
   text = pp.lowercase_unidecode(text) # converts text to lowercase and removes accents
   text = pp.remove_stopwords(text) # removes stopwords from the text
   text = pp.remove_tweet_marking(text) # removes hashtags from the text
   text = remove_repetion_caracteres(text) # reduce possible repetitions to a maximum of two occurrences
   text = pp.remove_urls(text) # removes URLs from the text
   text = pp.remove_punctuation(text) # removes punctuation from the text
   text = pp.remove_numbers(text) # removes or filters out numbers from the text
   text = pp.remove_n(text, n = 2) # removes words with length less than or equal to n from the text
   
   return text

In [8]:
df['clean_text'] = df['Agradeço a sua participação e abro o espaço para que você possa contribuir com alguma crítica, sugestão ou elogio sobre o Simulador de Aposentadoria.'].apply(preprocessing)
df

,ID,Você conhece e já utilizou o Simulador de Aposentadoria no SOUGOV.BR?,Acho que gostaria de utilizar o Simulador de Aposentadoria com frequência,1\nAcho que gostaria de utilizar o Simulador de Aposentadoria com frequência,Eu achei o Simulador de Aposentadoria muito complexo,2\nEu achei o Simulador de Aposentadoria muito complexo,Eu achei o Simulador de Aposentadoria fácil de usar,3\nEu achei o Simulador de Aposentadoria fácil de usar,Eu acho que precisaria de ajuda de uma pessoa com conhecimentos específicos para usar o Simulador de Aposentadoria,4\nEu acho que precisaria de ajuda de uma pessoa com conhecimentos específicos para usar o Simulador de Aposentadoria,...,8\nEu achei o Simulador de Aposentadoria difícil de usar,Eu me senti confiante usando o Simulador de Aposentadoria,9\nEu me senti confiante usando o Simulador de Aposentadoria,"Eu precisei aprender várias coisas novas antes de conseguir usar o Simulador de Aposentadoria. ,opcao:","10\nEu precisei aprender várias coisas novas antes de conseguir usar o Simulador de Aposentadoria. ,opcao:2","Agradeço a sua participação e abro o espaço para que você possa contribuir com alguma crítica, sugestão ou elogio sobre o Simulador de Aposentadoria.","Some a pontuação total de todas as questões ímpares (questões 1, 3, 5, 7 e 9) e, em seguida, subtraia 5 do total para obter (X).","Some a pontuação total de todas as questões de número par (questões 2, 4, 6, 8 e 10), então subtraia esse total de 25 para obter (Y).","Some a pontuação total dos novos valores (X+Y) e multiplique por 2,5.",clean_text
0,3,Sim,Concordo totalmente,5,Discordo totalmente,1,Concordo totalmente,5,Discordo totalmente,1,...,1,Discordo totalmente,1,Discordo totalmente,1,NaN,16,20,90,NaN
1,9,Sim,Discordo parcialmente,2,Concordo totalmente,5,Discordo totalmente,1,Concordo totalmente,5,...,5,Discordo totalmente,1,"Não concordo, nem discordo",3,NaN,1,4,"12,5",NaN
2,13,Sim,Concordo parcialmente,4,"Não concordo, nem discordo",3,"Não concordo, nem discordo",3,"Não concordo, nem discordo",3,...,4,Discordo parcialmente,2,Concordo totalmente,5,NaN,8,7,"37,5",NaN
3,15,Sim,Concordo parcialmente,4,Concordo parcialmente,4,Concordo parcialmente,4,Discordo parcialmente,2,...,5,Concordo totalmente,5,"Não concordo, nem discordo",3,NaN,17,10,"67,5",NaN
4,16,Sim,Concordo totalmente,5,Concordo parcialmente,4,Discordo parcialmente,2,Discordo totalmente,1,...,5,Concordo parcialmente,4,Discordo totalmente,1,"Ótima iniciativa, mas não funcionou comigo",12,9,"52,5",otimo iniciativa funcionar comigo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2250,4858,Sim,Concordo parcialmente,4,Concordo parcialmente,4,Discordo parcialmente,2,Concordo parcialmente,4,...,4,Discordo parcialmente,2,"Não concordo, nem discordo",3,Incluir um módulo relativo ao servidor com def...,8,7,"37,5",incluir modulo relativo deficiencia
2251,4859,Sim,"Não concordo, nem discordo",3,Concordo parcialmente,4,Discordo parcialmente,2,Concordo parcialmente,4,...,4,Discordo parcialmente,2,"Não concordo, nem discordo",3,NaN,7,7,35,NaN
2252,4860,Sim,Concordo totalmente,5,Discordo totalmente,1,Concordo totalmente,5,Discordo totalmente,1,...,1,Concordo totalmente,5,Discordo totalmente,1,NaN,20,20,100,NaN
2253,4861,Sim,Concordo parcialmente,4,Discordo parcialmente,2,"Não concordo, nem discordo",3,"Não concordo, nem discordo",3,...,2,"Não concordo, nem discordo",3,"Não concordo, nem discordo",3,NaN,11,14,"62,5",NaN


* **"Saving the changes made**

In [9]:
df.to_csv('../data/SUS_Simulador_Aposentadoria_pre_processado.csv', index=False)